<a href="https://colab.research.google.com/github/kedimonmuo/Keras-image_detection/blob/main/InceptionV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inception V3 Model Building for Sentiment Analysis

---



In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import layers, models
from keras.preprocessing.image import load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.applications import InceptionV3
import os

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"eceturgut","key":"78dbc486d9cfc36523e5b809fac83200"}'}

In [4]:
!pip install -q kaggle

In [5]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [6]:
!kaggle datasets download -d jonathanoheix/face-expression-recognition-dataset

 98% 118M/121M [00:00<00:00, 197MB/s] 
100% 121M/121M [00:00<00:00, 206MB/s]


In [7]:
zip_path = "/content/face-expression-recognition-dataset.zip"
# Extract from ZIP to avoid bugs

In [8]:
import zipfile

# Define the path to the uploaded ZIP file
zip_path = '/content/face-expression-recognition-dataset.zip'

# Define the directory where you want to extract the contents
extract_path = '/content/extracted_data/'

# Create the extraction directory if it doesn't exist
if not os.path.exists(extract_path):
    os.makedirs(extract_path)

# Extract the ZIP file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

 # Train and Test Data Augmentation

In [9]:
# Define sizeas and path variable
images = "/content/extracted_data/images"
image_size = (224, 224)  # Adjust based on input size expected by ResNet50
batch_size = 32
no_of_classes =7

#classes=['happy', 'sad', 'angry', 'fear', 'disgust', 'neutral', 'suprise']

# Data Augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
)

# Load and split the dataset
train_generator = train_datagen.flow_from_directory(
    images+ "/train",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')


validation_generator = ImageDataGenerator().flow_from_directory(
    images+ "/validation",
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')


Found 23060 images belonging to 7 classes.
Found 0 images belonging to 7 classes.


In [10]:
# Load pretrained InceptionV3 model
base_model = InceptionV3(weights='imagenet',
                        include_top=False,
                        input_shape=(224, 224, 3))

# Freeze the base model layers
base_model.trainable = False


87910968/87910968 [==============================] - 1s 0us/step


In [11]:
# Add custom layers for classification on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(7, activation='softmax')(x)  # 'number_of_classes' is the number of sentiment classes



In [12]:
# Create the final model
model = Model(inputs=base_model.input, outputs=predictions)

In [13]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss = "categorical_crossentropy",
              metrics=['accuracy'])

In [14]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)


In [ ]:
history = model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs=50,
                    callbacks=[early_stopping])

Epoch 1/50
721/721 [==============================] - ETA: 0s - loss: 1.5607 - accuracy: 0.4075

721/721 [==============================] - 290s 389ms/step - loss: 1.5607 - accuracy: 0.4075
Epoch 2/50
721/721 [==============================] - ETA: 0s - loss: 1.4171 - accuracy: 0.4545

721/721 [==============================] - 286s 396ms/step - loss: 1.4171 - accuracy: 0.4545
Epoch 3/50
721/721 [==============================] - ETA: 0s - loss: 1.3818 - accuracy: 0.4709

721/721 [==============================] - 284s 393ms/step - loss: 1.3818 - accuracy: 0.4709
Epoch 4/50
721/721 [==============================] - ETA: 0s - loss: 1.3526 - accuracy: 0.4831

721/721 [==============================] - 285s 395ms/step - loss: 1.3526 - accuracy: 0.4831
Epoch 5/50
721/721 [==============================] - ETA: 0s - loss: 1.3232 - accuracy: 0.4979

...

721/721 [==============================] - 269s 373ms/step - loss: 1.0014 - accuracy: 0.6229
Epoch 43/50
721/721 [==============================] - ETA: 0s - loss: 0.9972 - accuracy: 0.6301

721/721 [==============================] - 272s 376ms/step - loss: 0.9972 - accuracy: 0.6301
Epoch 44/50
721/721 [==============================] - ETA: 0s - loss: 0.9872 - accuracy: 0.6311

721/721 [==============================] - 300s 417ms/step - loss: 0.9872 - accuracy: 0.6311
Epoch 45/50
721/721 [==============================] - ETA: 0s - loss: 0.9846 - accuracy: 0.6366

721/721 [==============================] - 319s 443ms/step - loss: 0.9846 - accuracy: 0.6366
Epoch 46/50
721/721 [==============================] - ETA: 0s - loss: 0.9893 - accuracy: 0.6307

721/721 [==============================] - 318s 441ms/step - loss: 0.9893 - accuracy: 0.6307
Epoch 47/50
427/721 [================>.............] - ETA: 2:11 - loss: 0.9792 - accuracy: 0.6319

In [ ]:
# Model warn Apply :early_stopping(monitor ="loss"...)

In [ ]:
#Evaluate model metrics
val_loss, val_accuracy = model.evaluate(validation_generator)
train_loss, train_accuracy = model.evaluate(train_generator)

print(f"Validation Loss: {val_loss:.4f}")
print(f"Validation Accuracy: {val_accuracy*100:.2f}%")

In [ ]:
 #Runtime error to run other visual diagnostics.
 #Colab T GPU limit reached.